In [ ]:
import os
import nibabel as nib
import numpy as np
import surfa as sf 

def create_baseline_consensus(folder_path):
    # List of baseline files
    baseline_files = ['salehi.nii.gz', 'keraudren.nii.gz', 'ebner.nii.gz']

    # Load baseline masks
    masks = []
    for baseline_file in baseline_files:
        file_path = os.path.join(folder_path, baseline_file)
        if os.path.exists(file_path):
            mask_data = sf.load_volume(file_path)#nib.load(file_path).get_fdata()
            mask_data = mask_data.data.astype(np.int32)

            print(mask_data.shape)
            masks.append(mask_data)
        else:
            print(f"Warning: {baseline_file} not found in {folder_path}")

    if not masks:
        print("Error: No baseline masks found.")
        return

    # Combine masks using logical OR
    consensus_mask = np.logical_or.reduce(masks).astype(np.uint8)

    # Create nibabel image
    affine = nib.load(os.path.join(folder_path, baseline_files[0])).affine
    consensus_image = nib.Nifti1Image(consensus_mask, affine)

    # Save the consensus image
    consensus_file_path = os.path.join(folder_path, 'baseline-consensus.nii.gz')
    nib.save(consensus_image, consensus_file_path)

    print(f"Baseline consensus mask saved to: {consensus_file_path}")

# Example usage:
folder_path = 'test/haste_trim2/mom_040__19wk__4T2_Haste_Sag_mm_400_FOV'
create_baseline_consensus(folder_path)
